## <center>语言模型的系统学习</center>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
gpu_no = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
# 定义TensorFlow配置
config = tf.ConfigProto()
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# 在创建session的时候把config作为参数传进去
sess = tf.Session(config = config)

### 1 词向量
语言模型包括文法语言模型和统计语言模型,一般我们指的是统计语言模型. word embedding 的映射方法是建立在分布假说基础上的,即假设词的语义由其上下文决定,上下文相似的单词,其语义相似. 上一节中,所使用的词向量是 one_hot 编码,由于 one_hot 编码中所有的字都是独立的,所以该语言模型学到的词与词的上下文信息只能存放在网络节点中.

#### (1) 词向量训练
在神经网络训练的词嵌入中, 一般会将所有的 embedding 随机初始化,然后在训练过程中不断更新 embedding 矩阵的值. 对于每一个词与它对应的向量的映射值,在 Tensorflow 中使用了一个叫 $tf.nn.embedding\_lookup$ 的方法来完成. 举例如下:
```python
with tf.device("/cpu:0"):
    embedding = tf.get_variable("embedding",[vocab_size,vec_size])
    inputs = tf.nn.embedding_lookup(embedding,input_data)
```
上面代码事先定义了 embedding 有 vocab_size 个词,每个词的向量个数为 vec_size 个. 最终得到的 inputs 就是输入向量 input_data 映射好的词向量了. 比如 input_data 的形状为 [batch_size,n_word], 那么 inputs 就变成了 [batch_size,n_word,vec_size]

<font color="red">注意</font><br/>
由于词向量定义好之后需要在训练过程中优化,所以 embedding 类型必须是 $tf.Variable$ 并且 $trainable = True$, $embedding\_lookup$ 这个函数目前只支持在 cpu 上运行

#### (2) CBOW & Skip-Gram
训练词向量时,比较常见的组合是 Skip-Gram 和 负采样方法, 这种组合速度快,效果好. 通常来说 <font color="red">CBOW 要比 Skip-Gram 更快</font>
+ CBOW 也是统计语言模型的一种,顾名思义就是根据某个词前面的 $n$ 个词或者前后 $n$ 个连续的词,来计算某个词出现的概率.
+ Skip-Gram 模型与之相反,是根据某个词,然后分别计算它前后出现的某几个词的各个概率.

例如, "我爱人工智能" 对于 CBOW 模型来讲,首先将所有字转换成 one_hot, 然后取出其中的一个字当做输入,将其前面和后面的字分别当做标签,拆分成如下样子:

+ "我",  "爱"
+ "爱",  "我"
+ "爱",  "人"
+ "人",  "爱"
+ "人",  "工"

每一行代表一个样本,第一列代表输入,第二列代表标签.将输入数据送进神经网络,同时将输出与标签计算 loss.进行优化迭代,在整个词库中如果字数特别多,会产生很大的矩阵,影响 softmax 速度.

word2vec 使用基于 Huffman 编码的 Hierarchical softmax 筛选掉了一些不可能的词,然后又使用 negative sampling 再去除一些负养的词, 所以时间复杂度就从 $O(V)$ 变成了 $O(log V)$

#### (3) 在 Tensorflow 中的  word2vec

在 Tensorflow 中提供了几个候选采样函数,用来处理 loss 计算中候选采样的工作,他们按照不同的采样规则被封装成了不同的函数,说明如下:
```python
+ tf.nn.uniform_candidate_sampler: 均匀的采样出类别子集
+ tf.nn.log_uniform_candidate_sampler: 按照 log_uniform (Zipfian) 分布采样, 指只有少数词经常被使用
+ tf.nn.learned_unigram_candidate_sampler: 按照训练数据中出现的类别分布进行采样
+ tf.nn.fixed_unigram_candidate_sampler: 按照用户提供的概率分布进行采样
```
如果不知道样本的分布,可以使用 $tf.nn.learned\_unigram\_candidate\_sampler$. 它的做法是先初始化一个 [0,range_max] 的数组,数组元素初始值为 1, 在训练过程中,碰到一个类别,就将相应的数组元素加 1, 每次按照数组归一化得到的概率进行采样来实现.

<font color="red">注意</font><br/>
在语言相关的任务中,词按照出现频率从大到小排序之后,服从 Zipfian 分布. 一般会先对类别按照出现频率从大到小排序,然后使用 $tf.nn.log\_uniform\_candidate\_sampler$ 函数.

在 TEnsorflow 的 word2vec 实现里,对比目标样本的损失值,计算 softmax,负采样等过程统统封装到了 nce_loss 函数中, 其默认使用的是 $tf.log\_uniform\_candidate\_sampler$ 采样函数,在不指定特殊采样器时,在该函数实现中会把词频越大的词,其编号也定义的越大,<font color='red'>即优先采用词频高的词作为负样本,词频越高越有可能作为负样本</font>, nce_loss 函数配合优化器可以对最后一层的权重进行调优.

#### (4) nce_loss 函数
nce_loss 函数定义如下:
```python
 
```